# Advanced Simulation

## Imports
Importing required libraries (also applying future annotations for Python 3.8).

In [66]:
from __future__ import annotations
from typing import Any, Optional
from re import findall, sub

import numpy as np
import numpy.typing as npt
import pandas as pd

## Set constants
Set ```CURRENT_TIME``` to detect the current time, set ```FLOORS``` to set the number of floors in the building.

In [7]:
# Set current time
CURRENT_TIME: float = 0.0


# Set constants
FLOORS = 6
DWELL_TIME = 10.0
MAX_RUNTIME = 60.0
ELEVATOR_TRAVEL_TIME = 8.0

## Create State
Create a state class that keeps track of the current state. The state contains the current direction of the elevator (```False``` for down, ```True``` for up), the current floor, the state of the door (```False``` for closed, ```True``` for open), a matrix of people who are waiting and a vector that keeps track of people waiting in the elevator.

In [8]:
class State:
    direction: bool
    current_floor: int
    door: bool
    waiting: np.ndarray
    elevator: np.ndarray

    def __init__(self):
        self.direction = False # True: up, False: down
        self.current_floor = 5
        self.door = True # True: open, False: closed
        self.waiting = np.zeros((FLOORS, FLOORS), np.int_)
        self.elevator = np.zeros(FLOORS, np.int_)

    def copy(self) -> State:
        new_state = State()
        new_state.direction = self.direction
        new_state.current_floor = self.current_floor
        new_state.door = self.door
        new_state.waiting = self.waiting.copy()
        new_state.elevator = self.elevator.copy()
        return new_state

    def __str__(self) -> str:
        return f"{{ direction: {self.direction}, current_floor: {self.current_floor}, door: {self.door}, waiting: {self.waiting}, elevator: {self.elevator} }}"

    def __repr__(self) -> str:
        return self.__str__()

example_state = State()

## Get active events
This function returns a set of events that are currently active. It always returns the arrival events, if the door is open it returns the close door event, if it is not it returns the elevator arrival event. (Ik heb besloten om door open weg te laten, want anders kreeg ik een rare bug dat de deur soms random open ging).

In [9]:
# Get set of active events, FIXME events could probably be changed in some sort of class
def get_events(state: State) -> set[str]:
    events = set()

    for i in range(FLOORS):
        for j in range(FLOORS):
            if i == j:
                continue
            events.add(f"Arrival{i},{j}")

    # FIXME ik  moet nog wat bedenken voor DoorOpen, misschien laat ik die wel gewoon weg
    if state.door:
        events.add("DoorClose")
    else:
        events.add(f"ElevatorArrival{state.current_floor + (2*state.direction - 1)}") 
    
    return events

get_events(example_state)

{'Arrival0,1',
 'Arrival0,2',
 'Arrival0,3',
 'Arrival0,4',
 'Arrival0,5',
 'Arrival1,0',
 'Arrival1,2',
 'Arrival1,3',
 'Arrival1,4',
 'Arrival1,5',
 'Arrival2,0',
 'Arrival2,1',
 'Arrival2,3',
 'Arrival2,4',
 'Arrival2,5',
 'Arrival3,0',
 'Arrival3,1',
 'Arrival3,2',
 'Arrival3,4',
 'Arrival3,5',
 'Arrival4,0',
 'Arrival4,1',
 'Arrival4,2',
 'Arrival4,3',
 'Arrival4,5',
 'Arrival5,0',
 'Arrival5,1',
 'Arrival5,2',
 'Arrival5,3',
 'Arrival5,4',
 'DoorClose'}

## Transition function
The ```new_state``` function returns the new state in case an event happens. It has two parameters, the ```old_state``` and the ```event```. In case of an unknown event it raises an ```Exception```.

In [10]:
def new_state(old_state: State, event: str):
    if "Elevator" in event:
        return parse_elevator_arrival(old_state, event)
    elif "Arrival" in event:
        return parse_arrival(old_state, event)
    elif "DoorClose" in event:
        return parse_door_close(old_state, event)

    raise Exception("Unknown event")

# Called in case of an arrival
def parse_arrival(old_state: State, event: str):
    state = old_state.copy()

    # Get event arrival and target floor
    current_floor, target_floor = sub(r"[a-zA-Z]*", r"", event).split(",")
    current_floor = int(current_floor)
    target_floor = int(target_floor)

    # If elevator open and on current floor enter directly
    if current_floor == state.current_floor and state.door:
        state.elevator[target_floor] += 1

    # Else enter waiting
    else:
        state.waiting[current_floor, target_floor] += 1

    return state

def should_continue_in_direction(waiting: np.ndarray, elevator: np.ndarray, current_floor: int, direction: bool) -> bool:
    if direction is True:
        # people in direction = people waiting in floors in current direction + people in elevator wanting to go in current direction
        people_in_direction = waiting[current_floor:,:].sum() + elevator[current_floor:].sum()
    else:
        people_in_direction = waiting[:current_floor,:].sum() + elevator[:current_floor].sum()
    
    return people_in_direction > 0 # Returns true if there are people needing to go to current direction

# Called in case of elevator arrival
def parse_elevator_arrival(old_state: State, event: str):
    state = old_state.copy()
    arrival_floor = int(findall(r"[\d+]", event)[0])

    state.current_floor = arrival_floor
    
    # If calls open door, or if people wanting to leave
    if state.waiting[arrival_floor,:].sum() + state.elevator[arrival_floor] > 0:
        state.door = True # Open door
        state.elevator[arrival_floor] = 0 # Everyone who has to be at this level leaves
        state.elevator += state.waiting[arrival_floor, :] # FIXME Check for max 10 # Everyone at this level enters elevator
        state.waiting[arrival_floor,:] = 0 # Everyone who has entered the elevator is not waiting anymore

    # After arrival we need to set new direction (i.e. if we do not want to go up any more we go down etc.)
    
    # If at either limit, change direction
    if state.current_floor in {0, FLOORS - 1}:
        state.direction = not state.direction

    # If no more calls in this direction change direction
    elif should_continue_in_direction(state.waiting, state.elevator, state.current_floor, state.direction) is False:
        state.direction = not state.direction

    return state

# Called in case of door close
def parse_door_close(old_state: State, event: str):
    state = old_state.copy()

    state.door = False
    return state

## Simulate events
This function simulates the events that are currently active, these are stored in a Pandas DataFrame (i.e. some sort of Python table). All new events are generated, all old events are copied to the new row, then finally we return the table with all the currently scheduled events.

In [106]:
def simulate_new_events(new_events: set[str], old_events: Optional[set[str]] = None, simulation_times: Optional[pd.DataFrame] = None):
    # If we haven't yet created a schedule table, create one
    if simulation_times is None:
        simulation_times =  pd.DataFrame({}, index=[CURRENT_TIME])
    
    # Copy the table s.t. we do not accedentially overwrite the current one
    simulation_times = simulation_times.copy()

    # Simulate all new events, add them to the schedule table
    for event in new_events:
        simulation_times.loc[CURRENT_TIME, event] = CURRENT_TIME + simulate_event(event)

    # If there are old events copy them to the new row
    if old_events:
        simulation_times.loc[CURRENT_TIME, list(old_events)] = simulation_times.iloc[-2][list(old_events)]

    return simulation_times


# This function ensures that we generate the right data for the right event
def simulate_event(event: str) -> float:
    if "Elevator" in event:
        return simulate_elevator_arrival(event)
    elif "Arrival" in event:
        return simulate_arrival(event)
    elif "Door" in event:
        return simulate_door_close(event)

    raise Exception("Unrecognized event")

# The three functions below simulate the values
def simulate_arrival(event: str) -> float:
    # Voor nu simuleren we voor alles gewoon een arrival time van gemiddeld 5 seconden, dat slaat nog nergens op maar dan kon ik ff snel testen hoe snel het allemaal was
    return np.random.exponential(5)

def simulate_elevator_arrival(event: str) -> float:
    # For now we're just returning the travel time
    return ELEVATOR_TRAVEL_TIME

def simulate_door_close(event: str) -> float:
    # For now we're just returning the dwell time
    return DWELL_TIME

## Total simulation
This function runs the total simulation for the set dwell time.

In [115]:
def simulate_elevator(dwell_time: float):
    global CURRENT_TIME, DWELL_TIME
    CURRENT_TIME = 0.0
    DWELL_TIME = dwell_time
    state = State()
    active_events = get_events(state)
    scheduled_events = simulate_new_events(active_events)

    while CURRENT_TIME < MAX_RUNTIME: # Run for 60 seconds
        next_event = scheduled_events.idxmin(axis=1)[CURRENT_TIME] #type:ignore # Get next event #
        CURRENT_TIME = scheduled_events.loc[CURRENT_TIME, next_event] #type:ignore # Get new time
        state = new_state(state, next_event) # Get the new state
        new_active_events = get_events(state) # Get all events that are now active
        old_events = (active_events - {next_event}).intersection(new_active_events) # Get all events that were already active
        new_events = new_active_events - (active_events - {next_event}) # Get all new events
        scheduled_events = simulate_new_events(new_events, old_events, scheduled_events) # Add all new events to table
        active_events = new_active_events # Set active events are new active events

        print(f"Next event: {next_event}, at: {CURRENT_TIME}")
        print(f"New state: {state}\n")

simulate_elevator(10.0)

Next event: Arrival5,4, at: 0.0852264571370902
New state: { direction: False, current_floor: 5, door: True, waiting: [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]], elevator: [0 0 0 0 1 0] }

Next event: Arrival4,5, at: 0.21127767897644753
New state: { direction: False, current_floor: 5, door: True, waiting: [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]], elevator: [0 0 0 0 1 0] }

Next event: Arrival0,4, at: 0.5378359868299574
New state: { direction: False, current_floor: 5, door: True, waiting: [[0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]], elevator: [0 0 0 0 1 0] }

Next event: Arrival5,0, at: 0.6247421364246887
New state: { direction: False, current_floor: 5, door: True, waiting: [[0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]], elevator: [1 0 0 0 1 0] }

Next event: Arrival0,3, at: 0.9583631035510499
New stat